In [1]:


import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import glob
import os
import csv
from scipy.stats import f_oneway
from scipy.stats import shapiro, levene, probplot
import math
import matplotlib.pyplot as plt
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import sys
#conversion factors
in_to_m = 0.0254

def load_tensile_csv(csv_path):
    """
    Loads tensile raw data CSVs that may contain:
      - metadata header lines
      - mixed delimiters
      - inconsistent column counts
    Returns a cleaned DataFrame with numeric columns only.
    """

    # First: Detect delimiter
    with open(csv_path, "r") as f:
        sample = f.read(2048)

    try:
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(sample, delimiters=[',',';','\t',' '])
        delimiter = dialect.delimiter
    except:
        delimiter = ','  # fallback

    # Second: Read the file while skipping bad rows
    df = pd.read_csv(
        csv_path,
        sep=delimiter,
        engine="python",
        comment='#',
        skiprows=5,
        #on_bad_lines='skip',    # <-- prevents crash
        skip_blank_lines=True
    )

    # Third: Drop EMPTY or non-numeric columns
    for col in df.columns:
        try:
            df[col] = pd.to_numeric(df[col], errors='coerce')
        except:
            df[col] = pd.NA

    df = df.dropna(axis=1, how='all')  # remove columns with no data
    df = df.dropna(axis=0, how='any')  # remove unfinished rows

    print(df)
    return df

def compute_youngs_modulus(
    csv_path,
    gauge_length,
    cross_section_area,
    load_col='Load',
    disp_col='Extension',
    elastic_strain_max=0.005
):
    """
    Computes Young's modulus from tensile test data.

    Parameters
    ----------
    csv_path : str
        Path to CSV file containing load/displacement data.
    gauge_length : float
        Original specimen length (m).
    cross_section_area : float
        Cross-sectional area (m^2).
    load_col : str
        Column name for load/force.
    disp_col : str
        Column name for displacement/extension.
    elastic_strain_max : float
        Maximum strain to use for the linear elastic fit (default = 0.5%).

    Returns
    -------
    E : float
        Estimated Young’s modulus (Pa).
    stress : ndarray
    strain : ndarray
        Arrays of engineering stress and strain.
    """
    print(f"gauge_length {gauge_length}")
    print(f"x_area {cross_section_area}")

    # Load file
    df = load_tensile_csv(csv_path)

    # Extract data
    force = df[load_col].values
    disp  = df[disp_col].values

    # print(f"disp {disp}")

    # Engineering strain
    strain = disp / gauge_length

    # Engineering stress
    stress = force / cross_section_area

    # Select elastic region
    mask = strain <= elastic_strain_max
    strain_elastic = strain[mask].reshape(-1, 1)
    stress_elastic = stress[mask]

    # Linear regression (slope = E)
    # model = LinearRegression().fit(strain_elastic, stress_elastic)
    # E = model.coef_[0]
    E = 0

    return E, stress, strain


In [2]:
def ultimate_tensile_strength(stress):
    """
    Maximum stress achieved in the test.
    """
    return np.max(stress)

def offset_yield_strength(E, strain, stress, offset=0.02):
    """
    Computes the offset yield strength (default 2%).
    Uses interpolation to find intersection of stress-strain curve with
    offset line of slope equal to the elastic modulus.
    """
    offset_line = E*(strain - offset)

    # difference between actual curve and offset line
    diff = stress - offset_line
    
    # find zero crossing
    idx = np.where(np.diff(np.sign(diff)))[0]
    if len(idx) == 0:
        return None  # No yield point found
    
    i = idx[0]
    # linear interpolation for better accuracy
    x1, x2 = strain[i], strain[i+1]
    y1, y2 = diff[i], diff[i+1]
    
    strain_yield = x1 - y1*(x2 - x1)/(y2 - y1)
    stress_yield = np.interp(strain_yield, strain, stress)
    
    return stress_yield

def total_energy(strain, stress):
    """
    Integrates the stress-strain curve using the trapezoidal rule.
    Units depend on stress/strain units (e.g., MPa -> MJ/m^3).
    """
    return np.trapz(stress, strain)

def strain_at_fracture(strain):
    """
    Maximum strain recorded before failure.
    """
    return np.max(strain)

In [3]:
def analyze_tensile_folder(folder_path, gauge_length_list, cross_section_area_list, stress_func, strain_func):
    """
    Processes all CSV tensile data files in a folder and returns a summary table.
    """

    results = []

    csv_files = glob.glob(os.path.join(folder_path, "*.csv"))
    # print(csv_files)

    for csv_path in csv_files:
        try:

            sample_index = int(os.path.basename(csv_path).split("_")[2][0])-1
            # print(sample_index)
            # --- Stress & strain + Young's modulus ---
            E, stress_raw, strain_raw = compute_youngs_modulus(
                csv_path,
                gauge_length_list[sample_index],
                cross_section_area_list[sample_index]
            )
            # print("fail 1")


            stress = [stress_func(val) for val in stress_raw]
            strain = [strain_func(val) for val in strain_raw]

            # print(f"raw strain {strain_raw}")
            # print(f"raw stress {stress_raw}")

            # print(f" strain {strain}")
            # print(f" stress {stress}")

            # print(csv_path)
            
            # sys.exit()
            # --- Individual metrics ---
            uts = ultimate_tensile_strength(stress)
            # print("fail1.5")
            #ys  = offset_yield_strength(strain, stress, E)
            # print("fail1.5")
            energy = total_energy(strain, stress)
            # print("fail1.5")

            eps_f = strain_at_fracture(strain)
            # print("fail 2")
            results.append({
                "Filename": os.path.basename(csv_path),
                "Youngs_Modulus_Pa": E,
                "UTS_Pa": uts,
            #    "YieldStrength_2pct_Pa": ys,
                "TotalEnergy_Jperm3": energy,
                "StrainAtFracture": eps_f
            })

            # print(results, uts, energy, eps_f)

        except Exception as e:
            print(f"Error processing {csv_path}: {e}")

    return pd.DataFrame(results)


In [4]:
#Helper - ANOVA TEST
def run_anova_on_metric(results_dict, metric_col):
    """
    results_dict: dict[name -> DataFrame from analyze_tensile_folder]
    metric_col: string name of the column to test, e.g. "UTS_Pa"
    """
    group_values = []
    group_labels = []

    for condition_name, df in results_dict.items():
        # pull that column, drop NaN
        vals = df[metric_col].dropna().values

        # we need at least 2 samples for ANOVA to make sense
        if len(vals) >= 2:
            group_values.append(vals)
            group_labels.append(condition_name)
        else:
            print(f"Skipping {condition_name} for {metric_col} (only {len(vals)} sample(s))")

    if len(group_values) < 2:
        raise ValueError(f"Need at least 2 groups with data for ANOVA on {metric_col}")

    F, p = f_oneway(*group_values)

    print(f"\nANOVA for {metric_col}")
    print("Groups:", group_labels)
    print(f"F-statistic = {F:.4f}, p-value = {p:.4e}")

    return F, p, group_labels

In [ ]:
#Helper - Shapiro test
def run_shapiro_per_group(results_by_condition, metric_col):
    """
    Run Shapiro–Wilk test for normality in each group (condition) for one metric.
    """
    print(f"\n=== Shapiro–Wilk normality test for {metric_col} ===")
    shapiro_results = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values

        # Shapiro needs at least 3 points to be meaningful
        if len(vals) < 3:
            print(f"{condition_name}: n={len(vals)} (skipping, too few samples)")
            continue

        stat, p = shapiro(vals)
        shapiro_results.append({
            "Condition": condition_name,
            "n": len(vals),
            "W_stat": stat,
            "p_value": p
        })

        interpretation = "✅ approx. normal (fail to reject H0)" if p > 0.05 else "⚠️ non-normal (reject H0)"
        print(f"{condition_name}: n={len(vals)}, W={stat:.4f}, p={p:.4f} → {interpretation}")

    return pd.DataFrame(shapiro_results)


In [6]:
#Helper - Q-Q plots

def qq_plots_per_group(results_by_condition, metric_col):
    """
    Generate Q–Q plots for each group (condition) for one metric.
    """
    n_groups = len(results_by_condition)
    ncols = 3  # adjust if you want a different layout
    nrows = math.ceil(n_groups / ncols)

    fig, axes = plt.subplots(nrows, ncols, figsize=(4 * ncols, 4 * nrows))
    axes = np.array(axes).reshape(-1)  # flatten in case of 2D

    fig.suptitle(f"Q–Q plots for {metric_col}", fontsize=16)

    for ax, (condition_name, df) in zip(axes, results_by_condition.items()):
        vals = df[metric_col].dropna().values

        if len(vals) < 3:
            ax.set_title(f"{condition_name}\n(n={len(vals)}, skipped)")
            ax.axis("off")
            continue

        probplot(vals, dist="norm", plot=ax)
        ax.set_title(f"{condition_name} (n={len(vals)})")

    # Hide any extra axes if number of groups < nrows*ncols
    for j in range(len(results_by_condition), len(axes)):
        axes[j].axis("off")

    plt.tight_layout()
    plt.show()


In [7]:
#Helper - LEVENES test
def run_levene_across_groups(results_by_condition, metric_col, center='median'):
    """
    Run Levene’s test for homogeneity of variances across groups for one metric.
    center: 'median' (more robust) or 'mean'
    """
    group_values = []
    group_labels = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values
        if len(vals) < 2:
            print(f"{condition_name}: n={len(vals)} (skipping in Levene, too few samples)")
            continue
        group_values.append(vals)
        group_labels.append(condition_name)

    if len(group_values) < 2:
        raise ValueError(f"Need at least 2 groups with data for Levene’s test on {metric_col}")

    stat, p = levene(*group_values, center=center)

    interpretation = "✅ variances approx. equal (fail to reject H0)" if p > 0.05 else "⚠️ variances differ (reject H0)"

    print(f"\n=== Levene’s test for {metric_col} (center={center}) ===")
    print("Groups:", group_labels)
    print(f"Levene stat = {stat:.4f}, p = {p:.4f} → {interpretation}")

    return stat, p, group_labels


In [8]:
#Helper - Tukey HSD
def run_tukey_for_metric(results_by_condition, metric_col, alpha=0.05):
    """
    results_by_condition: dict[condition_name -> DataFrame from analyze_tensile_folder]
    metric_col: e.g. "UTS_Pa", "TotalEnergy_Jperm3", "StrainAtFracture"
    """
    all_values = []
    all_groups = []

    for condition_name, df in results_by_condition.items():
        vals = df[metric_col].dropna().values
        if len(vals) == 0:
            continue

        all_values.extend(vals)
        all_groups.extend([condition_name] * len(vals))

    all_values = np.array(all_values)
    all_groups = np.array(all_groups)

    tukey = pairwise_tukeyhsd(
        endog=all_values,   # the data
        groups=all_groups,  # group labels (folder names)
        alpha=alpha
    )

    print(f"\n=== Tukey HSD for {metric_col} (alpha = {alpha}) ===")
    print(tukey.summary())   # nice text table

    # Convert to DataFrame for easier viewing / export
    data = tukey._results_table.data
    header, rows = data[0], data[1:]
    tukey_df = pd.DataFrame(rows, columns=header)

    return tukey, tukey_df


In [9]:
#Helper - After Sinter Part Dimensino Dictionary
after_print_dim = pd.read_csv("after_sinter_dimensions_2.csv")
all_combinations = ["Gyroid_8", "Gyroid_6", "Gyroid_4", "Solid_8", "Solid_6", "Solid_4", "Triangular_8", "Triangular_6", "Triangular_4"]

measurement_types = ["length (in)", "thickness (in)", "width (in)", "mass (g)"]

combo_to_measurement = {}
for combination in all_combinations:
    sub_df = after_print_dim[(after_print_dim["infill_type"] == combination.split("_")[0].lower()) & (after_print_dim["wall_thickness"] == float(combination.split("_")[1]))]
    
    #extracts the array STRINGS
    len_s = sub_df[sub_df["measurement"] == measurement_types[0]]["part_means"].values[0]
    thic_s = sub_df[sub_df["measurement"] == measurement_types[1]]["part_means"].values[0]
    width_s = sub_df[sub_df["measurement"] == measurement_types[2]]["part_means"].values[0]
    mass_s = sub_df[sub_df["measurement"] == measurement_types[3]]["part_means"].values[0]

    #converts array strings to array
    len_arr = np.array([float(x)*in_to_m for x in len_s.strip("[]").split()])
    thic_arr = np.array([float(x)*in_to_m for x in thic_s.strip("[]").split()])
    width_arr = np.array([float(x)*in_to_m for x in width_s.strip("[]").split()])
    mass_arr = np.array([float(x) for x in mass_s.strip("[]").split()])


    combo_to_measurement[combination] = [len_arr, thic_arr*width_arr,mass_arr]

print(combo_to_measurement)
#each entry --> combinatino
#within each entry [list of average length means for each sample], [list of average cross sectional area for each sample], [list of average masses for each sample]

{'Gyroid_8': [array([0.11042142, 0.11042142, 0.1103884 , 0.1104265 ]), array([2.17018918e-05, 2.18103647e-05, 2.17774185e-05, 2.18388383e-05]), array([17.52 , 17.514, 17.51 , 17.51 ])], 'Gyroid_6': [array([0.11066018, 0.11053064, 0.11057382, 0.11054588]), array([2.17503863e-05, 2.18874081e-05, 2.19652786e-05, 2.20334398e-05]), array([16.04, 16.04, 16.04, 16.04])], 'Gyroid_4': [array([0.11077194, 0.11087354, 0.1106932 , 0.11063478]), array([2.17694621e-05, 2.19991495e-05, 2.22288692e-05, 2.18847516e-05]), array([14.73, 14.73, 14.73, 14.73])], 'Solid_8': [array([0.11142726, 0.11144758, 0.11146282, 0.1114806 ]), array([2.18838272e-05, 2.16172683e-05, 2.16206237e-05, 2.19557838e-05]), array([22.69, 22.69, 22.69, 22.68])], 'Solid_6': [array([0.11143996, 0.11147044, 0.11147552, 0.1114298 ]), array([2.18221066e-05, 2.15389569e-05, 2.15593658e-05, 2.15899568e-05]), array([22.55, 22.55, 22.55, 22.63])], 'Solid_4': [array([0.11146282, 0.11141202, 0.11142472, 0.1114298 ]), array([2.12045382e-05, 

In [10]:
#Helper - Measurement and Tensile Calibration

# ---------- helper: generic linear calibration fit ----------
def fit_linear_calibration(x_measured, y_true):
    """
    Fits y_true = a * x_measured + b  via least squares.
    Returns (a, b).
    """
    x = np.asarray(x_measured, dtype=float)
    y = np.asarray(y_true, dtype=float)
    a, b = np.polyfit(x, y, 1)
    return a, b

# ---------- 1. Displacement calibration ----------
disp_df = pd.read_csv("Calibration_Filtererd_Data/displacement_calibration.csv")

a_disp, b_disp = fit_linear_calibration(
    disp_df["instron_extension_in"],
    disp_df["true_extension_in"]
)

print(f"Displacement calibration:")
print(f"  x_true = {a_disp:.6f} * x_instron + {b_disp:.6f}")

def calibrate_displacement(x_instron):
    """Apply displacement calibration to a NumPy array / Series / scalar."""
    return a_disp * x_instron + b_disp


# ---------- 2. Load calibration ----------
load_df = pd.read_csv("Calibration_Filtererd_Data/load_calibration.csv")

# Drop any rows where instron_load_lbf is still empty
load_df = load_df.dropna(subset=["instron_load_lbf"])

a_load, b_load = fit_linear_calibration(
    load_df["instron_load_lbf"],
    load_df["true_load_lbf"]
)

print(f"\nLoad calibration:")
print(f"  F_true = {a_load:.6f} * F_instron + {b_load:.6f}")

def calibrate_load(F_instron):
    """Apply load calibration to a NumPy array / Series / scalar."""
    return a_load * F_instron + b_load



Displacement calibration:
  x_true = 0.998695 * x_instron + 0.000012

Load calibration:
  F_true = 1.007285 * F_instron + 1.150238


In [11]:
#Goal 1

#get relevant folers
folder_names = ["Gyroid_8", "Gyroid_6", "Gyroid_4", "Solid_8", "Solid_6", "Solid_4", "Triangular_8", "Triangular_6", "Triangular_4"]
folder_addendum = ".is_tens_RawData"
parent_dir = "3DPrint"
folder_names_final = [os.path.join(parent_dir, name + folder_addendum) for name in folder_names]

#define lengths and section areas
L = 1*in_to_m #in
A = 0.125*0.25*in_to_m**2 #in^2

#
actual_length = 4.349*in_to_m
actual_area = (0.125*0.25)*in_to_m**2

#getting all the references
results_by_condition = {}

for i, folder_name in enumerate(folder_names_final):
    # print(f"dict {combo_to_measurement[folder_names[i]]}")

    reference_tests = analyze_tensile_folder(
        folder_name,
        combo_to_measurement[folder_names[i]][0]/actual_length*L, combo_to_measurement[folder_names[i]][1]/actual_area*A,
        calibrate_load, calibrate_displacement
    )

    results_by_condition[folder_name] = reference_tests

#define metrics
metrics = [
    "UTS_Pa",              # ultimate tensile strength
    "TotalEnergy_Jperm3",  # energy absorbed
    "StrainAtFracture"     # elongation at break
]

#ANOVA
anova_results = []

for metric in metrics:
    F, p, groups = run_anova_on_metric(results_by_condition, metric)
    anova_results.append({
        "Metric": metric,
        "F_stat": F,
        "p_value": p
    })

print(anova_results)

#Shaprio
for metric in metrics:
    shapiro_df = run_shapiro_per_group(results_by_condition, metric)
    display(shapiro_df)

#levene's
for metric in metrics:
    run_levene_across_groups(results_by_condition, metric, center='median')

#Q-Q PLOT
# for metric in metrics:
#     qq_plots_per_group(results_by_condition, metric)

#Tukey
tukey_tables = {}

for metric in metrics:
    tukey_result, tukey_df = run_tukey_for_metric(results_by_condition, metric)
    tukey_tables[metric] = tukey_df
    display(tukey_df)



gauge_length 0.02539123936537135
x_area 2.1838838327397274e-05
       Time  Extension         Load
1     0.000    0.00030     -2.64829
2     0.100    0.00764     43.94266
3     0.200    0.01751    104.44289
4     0.300    0.02691    132.00453
5     0.400    0.03581    182.97757
..      ...        ...          ...
384  37.884    3.16041  14899.19434
385  37.886    3.16064  13925.96191
386  37.888    3.16084  12905.07715
387  37.890    3.16104  11863.46484
388  37.892    3.16127  10824.25000

[388 rows x 3 columns]
gauge_length 0.025390071280754194
x_area 2.1701891796129603e-05
       Time  Extension         Load
1     0.000    0.00000      0.93332
2     0.100    0.00403      2.96457
3     0.200    0.01341      7.14769
4     0.300    0.02283     80.13934
5     0.400    0.03210    169.43657
..      ...        ...          ...
364  35.826    2.98581  14494.72461
365  35.828    2.98601  13531.48340
366  35.830    2.98621  12524.41406
367  35.832    2.98641  11500.87891
368  35.834    2.9866

/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_95715/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_95715/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_95715/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_95715/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stre

,Condition,n,W_stat,p_value
0,3DPrint/Gyroid_8.is_tens_RawData,4,0.840724,0.197523
1,3DPrint/Gyroid_6.is_tens_RawData,4,0.978149,0.891078
2,3DPrint/Gyroid_4.is_tens_RawData,4,0.798090,0.098838
3,3DPrint/Solid_8.is_tens_RawData,4,0.837619,0.188550
4,3DPrint/Solid_6.is_tens_RawData,4,0.672329,0.005265
5,3DPrint/Solid_4.is_tens_RawData,4,0.799753,0.101777
6,3DPrint/Triangular_8.is_tens_RawData,4,0.856433,0.247682
7,3DPrint/Triangular_6.is_tens_RawData,4,0.942791,0.671417
8,3DPrint/Triangular_4.is_tens_RawData,4,0.985881,0.935533



=== Shapiro–Wilk normality test for TotalEnergy_Jperm3 ===
3DPrint/Gyroid_8.is_tens_RawData: n=4, W=0.9402, p=0.6557 → ✅ approx. normal (fail to reject H0)
3DPrint/Gyroid_6.is_tens_RawData: n=4, W=0.8074, p=0.1162 → ✅ approx. normal (fail to reject H0)
3DPrint/Gyroid_4.is_tens_RawData: n=4, W=0.9980, p=0.9938 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_8.is_tens_RawData: n=4, W=0.8534, p=0.2373 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_6.is_tens_RawData: n=4, W=0.9172, p=0.5212 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_4.is_tens_RawData: n=4, W=0.9337, p=0.6162 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_8.is_tens_RawData: n=4, W=0.9701, p=0.8424 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_6.is_tens_RawData: n=4, W=0.9859, p=0.9357 → ✅ approx. normal (fail to reject H0)
3DPrint/Triangular_4.is_tens_RawData: n=4, W=0.9373, p=0.6377 → ✅ approx. normal (fail to reject H0)


,Condition,n,W_stat,p_value
0,3DPrint/Gyroid_8.is_tens_RawData,4,0.940232,0.655740
1,3DPrint/Gyroid_6.is_tens_RawData,4,0.807394,0.116155
2,3DPrint/Gyroid_4.is_tens_RawData,4,0.998048,0.993821
3,3DPrint/Solid_8.is_tens_RawData,4,0.853368,0.237255
4,3DPrint/Solid_6.is_tens_RawData,4,0.917166,0.521176
5,3DPrint/Solid_4.is_tens_RawData,4,0.933675,0.616165
6,3DPrint/Triangular_8.is_tens_RawData,4,0.970142,0.842360
7,3DPrint/Triangular_6.is_tens_RawData,4,0.985911,0.935700
8,3DPrint/Triangular_4.is_tens_RawData,4,0.937264,0.637717


...

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,1.247374e+08,0.0000,1.061810e+08,1.432938e+08,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,1.975816e+08,0.0000,1.790253e+08,2.161380e+08,True
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,5.484821e+08,0.0000,5.299257e+08,5.670384e+08,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,5.397553e+08,0.0000,5.211989e+08,5.583116e+08,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,5.328350e+08,0.0000,5.142787e+08,5.513914e+08,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,3.797573e+06,0.9986,-1.475879e+07,2.235393e+07,False
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,1.041671e+08,0.0000,8.561070e+07,1.227234e+08,True
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,2.056662e+08,0.0000,1.871099e+08,2.242226e+08,True
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,7.284424e+07,0.0000,5.428788e+07,9.140060e+07,True
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,4.237447e+08,0.0000,4.051883e+08,4.423010e+08,True



=== Tukey HSD for TotalEnergy_Jperm3 (alpha = 0.05) ===
                                              Multiple Comparison of Means - Tukey HSD, FWER=0.05                                              
               group1                               group2                     meandiff      p-adj        lower              upper       reject
-----------------------------------------------------------------------------------------------------------------------------------------------
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_6.is_tens_RawData   17337879041.8194    0.0    8411661140.1209  26264096943.5179   True
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_8.is_tens_RawData   10548042035.8195 0.0119     1621824134.121   19474259937.518   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_4.is_tens_RawData   87401108271.8122    0.0   78474890370.1138  96327326173.5107   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_6.is_tens_RawData   893

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,1.733788e+10,0.0000,8.411661e+09,2.626410e+10,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,1.054804e+10,0.0119,1.621824e+09,1.947426e+10,True
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,8.740111e+10,0.0000,7.847489e+10,9.632733e+10,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,8.930499e+10,0.0000,8.037877e+10,9.823121e+10,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,8.858140e+10,0.0000,7.965518e+10,9.750762e+10,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,-1.077176e+10,0.0097,-1.969797e+10,-1.845539e+09,True
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,-9.694060e+08,1.0000,-9.895624e+09,7.956812e+09,False
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,8.455994e+09,0.0737,-4.702243e+08,1.738221e+10,False
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-6.789837e+09,0.2494,-1.571605e+10,2.136381e+09,False
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,7.006323e+10,0.0000,6.113701e+10,7.898945e+10,True



=== Tukey HSD for StrainAtFracture (alpha = 0.05) ===
                               Multiple Comparison of Means - Tukey HSD, FWER=0.05                                
               group1                               group2                meandiff p-adj   lower    upper   reject
------------------------------------------------------------------------------------------------------------------
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_6.is_tens_RawData   17.951 0.0088   3.2157  32.6862   True
    3DPrint/Gyroid_4.is_tens_RawData     3DPrint/Gyroid_8.is_tens_RawData  -0.4219    1.0 -15.1572  14.3133  False
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_4.is_tens_RawData  56.9384    0.0  42.2031  71.6736   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_6.is_tens_RawData  55.1386    0.0  40.4033  69.8738   True
    3DPrint/Gyroid_4.is_tens_RawData      3DPrint/Solid_8.is_tens_RawData  56.4965    0.0  41.7613  71.2318   True
    3DPrint/Gyroid_4.is_t

,group1,group2,meandiff,p-adj,lower,upper,reject
0,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_6.is_tens_RawData,17.9510,0.0088,3.2157,32.6862,True
1,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-0.4219,1.0000,-15.1572,14.3133,False
2,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,56.9384,0.0000,42.2031,71.6736,True
3,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,55.1386,0.0000,40.4033,69.8738,True
4,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,56.4965,0.0000,41.7613,71.2318,True
5,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_4.is_tens_RawData,-27.5061,0.0000,-42.2413,-12.7708,True
6,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_6.is_tens_RawData,-16.2896,0.0220,-31.0248,-1.5543,True
7,3DPrint/Gyroid_4.is_tens_RawData,3DPrint/Triangular_8.is_tens_RawData,-7.8928,0.6804,-22.6281,6.8424,False
8,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Gyroid_8.is_tens_RawData,-18.3729,0.0069,-33.1081,-3.6376,True
9,3DPrint/Gyroid_6.is_tens_RawData,3DPrint/Solid_4.is_tens_RawData,38.9874,0.0000,24.2521,53.7227,True


In [12]:
#Goal 4

#get relevant folers
folder_names = [ "Solid_8", "Solid_6", "Solid_4"]
folder_addendum = ".is_tens_RawData"
parent_dir = "3DPrint"
folder_names_final = [os.path.join(parent_dir, name + folder_addendum) for name in folder_names] 
folder_names_final = folder_names_final + ["./Reference Tests/Reference_T1"]

#define lengths and section areas
L = 1*in_to_m #in
A = 0.125*0.25*in_to_m**2 #in^2

actual_length = 4.349*in_to_m
actual_area = (0.125*0.25)*in_to_m**2

#getting all the references
results_by_condition = {}

for i, folder_name in enumerate(folder_names_final):

    #use specific dimensions
    if not "Reference" in folder_name: 
        reference_tests = analyze_tensile_folder(
            folder_name,
            combo_to_measurement[folder_names[i]][0]/actual_length*L, combo_to_measurement[folder_names[i]][1]/actual_area*A,
            calibrate_load, calibrate_displacement
        )
    #USE CAD DIMENSIONS (for waterjet parts)
    else:
        reference_tests = analyze_tensile_folder(
            folder_name,
            [L]*6, [A]*6,
            calibrate_load, calibrate_displacement)
        
    results_by_condition[folder_name] = reference_tests

#define metrics
metrics = [
    "UTS_Pa",              # ultimate tensile strength
    "TotalEnergy_Jperm3",  # energy absorbed
    "StrainAtFracture"     # elongation at break
]

#ANOVA
anova_results = []

for metric in metrics:
    F, p, groups = run_anova_on_metric(results_by_condition, metric)
    anova_results.append({
        "Metric": metric,
        "F_stat": F,
        "p_value": p
    })

print(anova_results)

#Shaprio
for metric in metrics:
    shapiro_df = run_shapiro_per_group(results_by_condition, metric)
    display(shapiro_df)

#levene's
for metric in metrics:
    run_levene_across_groups(results_by_condition, metric, center='median')

#Q-Q PLOT
# for metric in metrics:
#     qq_plots_per_group(results_by_condition, metric)

#Tukey
tukey_tables = {}

for metric in metrics:
    tukey_result, tukey_df = run_tukey_for_metric(results_by_condition, metric)
    tukey_tables[metric] = tukey_df
    display(tukey_df)



gauge_length 0.025633616923430674
x_area 2.1955783771570437e-05
       Time  Extension         Load
1     0.000    0.00000      3.47343
2     0.100    0.00456      4.32416
3     0.200    0.01381     10.19229
4     0.300    0.02301     57.88463
5     0.400    0.03223    138.46074
..      ...        ...          ...
511  50.572    4.21466  20361.54297
512  50.574    4.21489  19005.10938
513  50.576    4.21511  17587.25586
514  50.578    4.21532  16151.84570
515  50.580    4.21557  14723.18164

[515 rows x 3 columns]
gauge_length 0.02562135203495056
x_area 2.18838272e-05
       Time  Extension         Load
1     0.000    0.00000     39.61577
2     0.100    0.00428     39.44108
3     0.200    0.01358     45.92258
4     0.300    0.02291    121.67086
5     0.400    0.03218    232.67726
..      ...        ...          ...
591  58.536    4.87829  20049.78125
592  58.538    4.87851  18733.70508
593  58.540    4.87874  17351.25586
594  58.542    4.87897  15942.98633
595  58.544    4.87922  14539

/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_95715/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_95715/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_95715/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stress, strain)
/var/folders/kw/6h6rqb2d1y90cxs7kdn3lf3r0000gn/T/ipykernel_95715/1993934651.py:38: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  return np.trapz(stre

,Condition,n,W_stat,p_value
0,3DPrint/Solid_8.is_tens_RawData,4,0.837619,0.188550
1,3DPrint/Solid_6.is_tens_RawData,4,0.672329,0.005265
2,3DPrint/Solid_4.is_tens_RawData,4,0.799753,0.101777
3,./Reference Tests/Reference_T1,4,0.943510,0.675840



=== Shapiro–Wilk normality test for TotalEnergy_Jperm3 ===
3DPrint/Solid_8.is_tens_RawData: n=4, W=0.8534, p=0.2373 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_6.is_tens_RawData: n=4, W=0.9172, p=0.5212 → ✅ approx. normal (fail to reject H0)
3DPrint/Solid_4.is_tens_RawData: n=4, W=0.9337, p=0.6162 → ✅ approx. normal (fail to reject H0)
./Reference Tests/Reference_T1: n=4, W=0.8671, p=0.2864 → ✅ approx. normal (fail to reject H0)


,Condition,n,W_stat,p_value
0,3DPrint/Solid_8.is_tens_RawData,4,0.853368,0.237255
1,3DPrint/Solid_6.is_tens_RawData,4,0.917166,0.521176
2,3DPrint/Solid_4.is_tens_RawData,4,0.933675,0.616165
3,./Reference Tests/Reference_T1,4,0.867094,0.286439


...

,group1,group2,meandiff,p-adj,lower,upper,reject
0,./Reference Tests/Reference_T1,3DPrint/Solid_4.is_tens_RawData,2.556621e+07,0.0070,7.020028e+06,4.411240e+07,True
1,./Reference Tests/Reference_T1,3DPrint/Solid_6.is_tens_RawData,1.683943e+07,0.0797,-1.706758e+06,3.538561e+07,False
2,./Reference Tests/Reference_T1,3DPrint/Solid_8.is_tens_RawData,9.919156e+06,0.4207,-8.627028e+06,2.846534e+07,False
3,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,-8.726786e+06,0.5243,-2.727297e+07,9.819398e+06,False
4,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-1.564706e+07,0.1096,-3.419324e+07,2.899128e+06,False
5,3DPrint/Solid_6.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-6.920270e+06,0.6918,-2.546645e+07,1.162591e+07,False



=== Tukey HSD for TotalEnergy_Jperm3 (alpha = 0.05) ===
                                        Multiple Comparison of Means - Tukey HSD, FWER=0.05                                        
             group1                          group2                  meandiff     p-adj        lower             upper       reject
-----------------------------------------------------------------------------------------------------------------------------------
 ./Reference Tests/Reference_T1 3DPrint/Solid_4.is_tens_RawData -32035396268.1692    0.0  -43213886286.842 -20856906249.4965   True
 ./Reference Tests/Reference_T1 3DPrint/Solid_6.is_tens_RawData -30131516487.6677    0.0 -41310006506.3405  -18953026468.995   True
 ./Reference Tests/Reference_T1 3DPrint/Solid_8.is_tens_RawData -30855104215.2739    0.0 -42033594233.9467 -19676614196.6011   True
3DPrint/Solid_4.is_tens_RawData 3DPrint/Solid_6.is_tens_RawData   1903879780.5015 0.9562  -9274610238.1712  13082369799.1743  False
3DPrint/Solid_4.is_

,group1,group2,meandiff,p-adj,lower,upper,reject
0,./Reference Tests/Reference_T1,3DPrint/Solid_4.is_tens_RawData,-3.203540e+10,0.0000,-4.321389e+10,-2.085691e+10,True
1,./Reference Tests/Reference_T1,3DPrint/Solid_6.is_tens_RawData,-3.013152e+10,0.0000,-4.131001e+10,-1.895303e+10,True
2,./Reference Tests/Reference_T1,3DPrint/Solid_8.is_tens_RawData,-3.085510e+10,0.0000,-4.203359e+10,-1.967661e+10,True
3,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,1.903880e+09,0.9562,-9.274610e+09,1.308237e+10,False
4,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,1.180292e+09,0.9888,-9.998198e+09,1.235878e+10,False
5,3DPrint/Solid_6.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-7.235877e+08,0.9973,-1.190208e+10,1.045490e+10,False



=== Tukey HSD for StrainAtFracture (alpha = 0.05) ===
                          Multiple Comparison of Means - Tukey HSD, FWER=0.05                           
             group1                          group2             meandiff p-adj   lower    upper   reject
--------------------------------------------------------------------------------------------------------
 ./Reference Tests/Reference_T1 3DPrint/Solid_4.is_tens_RawData -58.1048    0.0 -77.1062 -39.1034   True
 ./Reference Tests/Reference_T1 3DPrint/Solid_6.is_tens_RawData -59.9046    0.0 -78.9059 -40.9032   True
 ./Reference Tests/Reference_T1 3DPrint/Solid_8.is_tens_RawData -58.5466    0.0  -77.548 -39.5452   True
3DPrint/Solid_4.is_tens_RawData 3DPrint/Solid_6.is_tens_RawData  -1.7998 0.9918 -20.8012  17.2016  False
3DPrint/Solid_4.is_tens_RawData 3DPrint/Solid_8.is_tens_RawData  -0.4418 0.9999 -19.4432  18.5595  False
3DPrint/Solid_6.is_tens_RawData 3DPrint/Solid_8.is_tens_RawData   1.3579 0.9964 -17.6435  20.3593  False


,group1,group2,meandiff,p-adj,lower,upper,reject
0,./Reference Tests/Reference_T1,3DPrint/Solid_4.is_tens_RawData,-58.1048,0.0000,-77.1062,-39.1034,True
1,./Reference Tests/Reference_T1,3DPrint/Solid_6.is_tens_RawData,-59.9046,0.0000,-78.9059,-40.9032,True
2,./Reference Tests/Reference_T1,3DPrint/Solid_8.is_tens_RawData,-58.5466,0.0000,-77.5480,-39.5452,True
3,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_6.is_tens_RawData,-1.7998,0.9918,-20.8012,17.2016,False
4,3DPrint/Solid_4.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,-0.4418,0.9999,-19.4432,18.5595,False
5,3DPrint/Solid_6.is_tens_RawData,3DPrint/Solid_8.is_tens_RawData,1.3579,0.9964,-17.6435,20.3593,False


#GOAL 1
